In [ ]:
import hydra
import torch
from tqdm.notebook import tqdm
from lib.utils.config import load_config
from lib.optimizer.framework import NeuralOptimizer
from lib.data.loader import load_intrinsics
from lib.data.loader import load_intrinsics
from lib.rasterizer import Rasterizer
from lib.renderer.renderer import Renderer
from lib.renderer.camera import Camera
from lib.tracker.timer import TimeTracker
from collections import defaultdict
import pandas as pd


def path_to_abblation(path):
    return "_".join(path.split("/")[-3].split("_")[1:])


def eval_iterations(optimizer, datamodule, N: int = 3):
    optimizer.max_iters = N
    optimizer.max_optims = 1
    time_tracker = TimeTracker()
    n_threshold =  optimizer.c_module.n_threshold
    d_threshold = optimizer.c_module.d_threshold
    # initial evaluation no optimization
    p_loss = []
    v_loss = []
    g_loss = []
    for batch in datamodule.val_dataloader():
        with torch.no_grad():
            batch = optimizer.transfer_batch_to_device(batch, "cuda", 0)
            time_tracker.start("optimize")
            out = optimizer(batch)
            time_tracker.stop("optimize")
            optimizer.c_module.n_threshold = -1.0
            optimizer.c_module.d_threshold = 1e-02  # 1cm
            loss_info = optimizer.compute_loss(batch=batch, out=out)
            optimizer.c_module.n_threshold = n_threshold
            optimizer.c_module.d_threshold = d_threshold  # 1cm
            p_loss.append(loss_info["loss_param"])
            v_loss.append(loss_info["loss_vertices"])
            g_loss.append(loss_info["loss_geometric_point2point"])
    iters_p_loss = torch.stack(p_loss).mean().item()
    iters_g_loss = torch.stack(g_loss).mean().item()
    iters_v_loss = torch.stack(v_loss).mean().item()
    t_perf = [torch.tensor(t.time_ms) for t in list(time_tracker.tracks.values())[0]]
    iters_time = torch.stack(t_perf).min().item()
    return iters_p_loss, iters_g_loss, iters_v_loss, iters_time


def load_flame_renderer():
    # instanciate similar to training
    cfg = load_config("train", ["data=kinect"])
    K = load_intrinsics(data_dir=cfg.data.intrinsics_dir, return_tensor="pt")
    camera = Camera(
        K=K,
        width=cfg.data.width,
        height=cfg.data.height,
        near=cfg.data.near,
        far=cfg.data.far,
        scale=cfg.data.scale,
    )
    rasterizer = Rasterizer(width=camera.width, height=camera.height)
    renderer = Renderer(rasterizer=rasterizer, camera=camera)
    flame = hydra.utils.instantiate(cfg.model)
    return flame, renderer


def load_neural_optimizer(flame, renderer, path, override=[]):
    o = ["data=kinect"] + override
    cfg = load_config("train", o)
    correspondence = hydra.utils.instantiate(cfg.correspondence)
    weighting = hydra.utils.instantiate(cfg.weighting)
    residuals = hydra.utils.instantiate(cfg.residuals)
    regularize = hydra.utils.instantiate(cfg.regularize)
    neural_optimizer = NeuralOptimizer.load_from_checkpoint(
        path,
        renderer=renderer,
        flame=flame,
        correspondence=correspondence,
        regularize=regularize,
        residuals=residuals,
        weighting=weighting,
    )
    return neural_optimizer


def load_icp_optimizer(flame, renderer, overrides):
    o = ["data=kinect", "optimizer.output_dir=none"] + overrides
    cfg = load_config("train", o)
    correspondence = hydra.utils.instantiate(cfg.correspondence)
    weighting = hydra.utils.instantiate(cfg.weighting)
    residuals = hydra.utils.instantiate(cfg.residuals)
    optimizer = hydra.utils.instantiate(cfg.optimizer)
    regularize = hydra.utils.instantiate(cfg.regularize)
    icp_optimizer = hydra.utils.instantiate(
        cfg.framework,
        flame=flame,
        logger=None,
        renderer=renderer,
        correspondence=correspondence,
        regularize=regularize,
        residuals=residuals,
        optimizer=optimizer,
        weighting=weighting,
    )
    return icp_optimizer.to("cuda")


# setup the datamodule
def load_datamodule(renderer, start_frame, end_frame, jump_size):
    cfg = load_config("train", ["data=kinect"])
    datamodule = hydra.utils.instantiate(
        cfg.data,
        renderer=renderer,
        val_dataset=dict(
            start_frame=start_frame,
            end_frame=end_frame,
            jump_size=jump_size,
        ),
    )
    datamodule.setup("fit")
    return datamodule

In [ ]:
N = 2
step_size = 0.7
# start_frame = 48 
# end_frame = 56
start_frame = 30
end_frame = None

# checkpoints
# ours = "/home/borth/GuidedResearch/checkpoints/kinect/ours.ckpt"
# ours_wo_prior = "/home/borth/GuidedResearch/checkpoints/kinect/wo_prior.ckpt"
# ours_syn = "/home/borth/GuidedResearch/checkpoints/kinect/synthetic.ckpt"
ours = "/home/borth/GuidedResearch/logs/2024-10-17/19-21-35_train_kinect_fix/checkpoints/epoch_379.ckpt"
ours_syn = "/home/borth/GuidedResearch/logs/2024-10-17/19-21-35_train_synthetic/checkpoints/epoch_1079.ckpt"

# loadings
times = defaultdict(dict)
p_losses = defaultdict(dict)
g_losses = defaultdict(dict)
v_losses = defaultdict(dict)
flame, renderer = load_flame_renderer()

for jump_size in [1, 2, 4, 8]:
    datamodule = load_datamodule(renderer, start_frame, end_frame, jump_size)

    # override = ["residuals=face2face_wo_landmarks", "regularize=dummy"]
    # optimizer = load_neural_optimizer(flame, renderer, ours_wo_prior, override)
    # optimizer.optimizer.step_size = step_size
    # p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    # key = "ours_wo_prior"
    # times[key][jump_size] = time
    # p_losses[key][jump_size] = p_loss
    # v_losses[key][jump_size] = v_loss
    # g_losses[key][jump_size] = g_loss    

    optimizer = load_neural_optimizer(flame, renderer, ours_syn, [])
    optimizer.optimizer.step_size = step_size
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "ours_syn"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

    optimizer = load_neural_optimizer(flame, renderer, ours, [])
    optimizer.optimizer.step_size = step_size
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "ours"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

    optimizer = load_icp_optimizer(flame, renderer, [])
    optimizer.optimizer.step_size = 0.0
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=1)
    key = "base"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

    optimizer = load_icp_optimizer(flame, renderer, ["residuals=point2plane", "weighting=dummy", "regularize=dummy"])
    optimizer.optimizer.step_size = step_size
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "icp-wo_reg"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

    optimizer = load_icp_optimizer(flame, renderer, ["residuals=face2face_wo_landmarks", "weighting=dummy", "regularize=dummy"])
    optimizer.optimizer.step_size = step_size
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "icp-w_reg"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

    optimizer = load_icp_optimizer(flame, renderer, ["residuals=face2face_wo_landmarks", "weighting=dummy", "regularize=dummy"])
    optimizer.optimizer.step_size = 0.3
    p_loss, g_loss, v_loss, time = eval_iterations(optimizer, datamodule, N=N)
    key = "icp-step"
    times[key][jump_size] = time
    p_losses[key][jump_size] = p_loss
    v_losses[key][jump_size] = v_loss
    g_losses[key][jump_size] = g_loss

In [ ]:
desired_order = ["base", "icp-w_reg", "icp-step", "icp-wo_reg", "ours", "ours_wo_prior", "ours_syn"]

# Create the DataFrame for p_losses
p_losses_df = pd.DataFrame(p_losses).transpose()
p_losses_df.columns = [f"0->{c}" for c in p_losses_df.columns]
p_losses_df = p_losses_df.reindex(desired_order)
p_losses_df.columns = pd.MultiIndex.from_tuples([("FLAME (norm)", c) for c in p_losses_df.columns])

# Create the DataFrame for g_losses
g_losses_df = pd.DataFrame(g_losses).transpose()
g_losses_df.columns = [f"0->{c}" for c in g_losses_df.columns]
g_losses_df = g_losses_df.reindex(desired_order)
g_losses_df.columns = pd.MultiIndex.from_tuples([("P2P (mm)", c) for c in g_losses_df.columns])

# Create the DataFrame for v_losses
v_losses_df = pd.DataFrame(v_losses).transpose()
v_losses_df.columns = [f"0->{c}" for c in v_losses_df.columns]
v_losses_df = v_losses_df.reindex(desired_order)
v_losses_df.columns = pd.MultiIndex.from_tuples([("Vertices (mm)", c) for c in v_losses_df.columns])

# Time
time_df = pd.DataFrame(times).transpose()
time_df = pd.DataFrame(time_df.mean(axis=1))
time_df = time_df.reindex(desired_order)
time_df.columns = pd.MultiIndex.from_tuples([("Time (ms)", "")])

# Concatenate the two DataFrames
pd.concat([p_losses_df, g_losses_df, v_losses_df, time_df], axis=1)